In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.1)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.91 sec - Loss 0.060906 - ACC 62.10% - ACC Mean 62.10% - AUC 66.76% - AUC Mean 66.76% - Deter 000
Ite 00050 - 1.65 sec - Loss 0.012443 - ACC 61.80% - ACC Mean 61.33% - AUC 67.45% - AUC Mean 65.99% - Deter 048
Ite 00052 - 1.58 sec - Loss 0.012063 - ACC 60.70% - ACC Mean 61.29% - AUC 65.42% - AUC Mean 65.92% - Deter 050
Early stop ite 52, rollback to correction of ite 2, whith acc of 63.75% and auc of 68.11%
OUR METHOD RUN 1/10 - acc: 63.75% +- 0.0% - auc: 68.11% +- 0.0%
Ite 00000 - 1.99 sec - Loss 0.140771 - ACC 62.40% - ACC Mean 62.40% - AUC 66.40% - AUC Mean 66.40% - Deter 000
Ite 00050 - 1.81 sec - Loss 0.053656 - ACC 61.80% - ACC Mean 60.86% - AUC 66.53% - AUC Mean 65.62% - Deter 002
Ite 00100 - 1.63 sec - Loss 0.035053 - ACC 59.00% - ACC Mean 60.85% - AUC 64.20% - AUC Mean 65.71% - Deter 007
Ite 00143 - 1.52 sec - Loss 0.026882 - ACC 60.10% - ACC Mean 61.25% - AUC 63.87% - AUC Mean 65.96% - Deter 050
Early stop ite 143, rollback to correction of ite 93, whith acc of 62

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 58.8% +- 0.0% - auc: 62.58% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 62.75% +- 2.33% - auc: 67.45% +- 1.69%
SFIL RUN 2/10 - acc: 61.2% +- 2.0% - auc: 66.17% +- 2.66%
SFIL RUN 3/10 - acc: 60.75% +- 3.48% - auc: 64.3% +- 4.17%
SFIL RUN 4/10 - acc: 61.9% +- 3.79% - auc: 66.08% +- 4.09%
SFIL RUN 5/10 - acc: 59.7% +- 0.48% - auc: 63.37% +- 1.91%
SFIL RUN 6/10 - acc: 60.75% +- 2.63% - auc: 65.32% +- 2.31%
SFIL RUN 7/10 - acc: 61.55% +- 2.74% - auc: 64.68% +- 3.44%
SFIL RUN 8/10 - acc: 60.8% +- 2.65% - auc: 66.17% +- 3.19%
SFIL RUN 9/10 - acc: 62.05% +- 2.01% - auc: 66.76% +- 2.55%
SFIL RUN 10/10 - acc: 61.5% +- 2.19% - auc: 65.46% +- 2.75%
SFIL GLOBAL - acc: 61.3% +- 0.81% - auc: 65.58% +- 1.15%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 59.05% +- 2.98% - auc: 63.79% +- 3.07%
PFIL RUN 2/10 - acc: 60.65% +- 2.24% - auc: 64.19% +- 1.6%
PFIL RUN 3/10 - acc: 59.35% +- 2.37% - auc: 62.08% +- 2.49%
PFIL RUN 4/10 - acc: 60.95% +- 2.98% - auc: 63.81% +- 3.37%
PFIL RUN 5/10 - acc: 59.1% +- 2.18% - auc: 63.47% +- 2.47%
PFIL RUN 6/10 - acc: 60.6% +- 1.81% - auc: 64.67% +- 1.28%
PFIL RUN 7/10 - acc: 58.85% +- 1.71% - auc: 63.78% +- 2.43%
PFIL RUN 8/10 - acc: 60.75% +- 1.34% - auc: 65.84% +- 1.01%
PFIL RUN 9/10 - acc: 60.75% +- 1.42% - auc: 65.4% +- 1.53%
PFIL RUN 10/10 - acc: 59.8% +- 1.76% - auc: 65.04% +- 2.54%
PFIL GLOBAL - acc: 59.98% +- 0.79% - auc: 64.21% +- 1.03%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 61.0% +- 1.65% - auc: 65.1% +- 3.18%
SPOL RUN 2/10 - acc: 59.55% +- 3.23% - auc: 64.22% +- 4.42%
SPOL RUN 3/10 - acc: 61.3% +- 0.99% - auc: 65.54% +- 0.67%
SPOL RUN 4/10 - acc: 61.45% +- 1.84% - auc: 66.63% +- 2.49%
SPOL RUN 5/10 - acc: 61.25% +- 2.5% - auc: 65.18% +- 2.39%
SPOL RUN 6/10 - acc: 58.85% +- 2.68% - auc: 63.26% +- 2.17%
SPOL RUN 7/10 - acc: 60.35% +- 1.71% - auc: 64.93% +- 2.17%
SPOL RUN 8/10 - acc: 60.3% +- 0.66% - auc: 63.97% +- 1.19%
SPOL RUN 9/10 - acc: 59.95% +- 2.13% - auc: 64.36% +- 2.0%
SPOL RUN 10/10 - acc: 61.45% +- 2.32% - auc: 65.99% +- 2.92%
SPOL GLOBAL - acc: 60.54% +- 0.85% - auc: 64.92% +- 0.95%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 59.75% +- 2.64% - auc: 63.44% +- 2.62%
PPOL RUN 2/10 - acc: 61.35% +- 3.2% - auc: 65.36% +- 3.47%
PPOL RUN 3/10 - acc: 59.4% +- 3.23% - auc: 64.98% +- 4.02%
PPOL RUN 4/10 - acc: 61.4% +- 3.04% - auc: 64.75% +- 3.49%
PPOL RUN 5/10 - acc: 61.7% +- 2.36% - auc: 65.56% +- 1.04%
PPOL RUN 6/10 - acc: 59.3% +- 2.5% - auc: 63.85% +- 3.43%
PPOL RUN 7/10 - acc: 59.5% +- 1.82% - auc: 63.98% +- 2.19%
PPOL RUN 8/10 - acc: 61.5% +- 1.8% - auc: 65.7% +- 2.56%
PPOL RUN 9/10 - acc: 60.7% +- 0.97% - auc: 64.64% +- 1.35%
PPOL RUN 10/10 - acc: 60.05% +- 2.98% - auc: 64.45% +- 3.01%
PPOL GLOBAL - acc: 60.46% +- 0.92% - auc: 64.67% +- 0.72%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
